# MLlib

Spark biedt ook een framework aan voor MachineLearning modellen te trainen op gedistribueerde datasets.
Dit framework is MLlib of ook wel sparkML genoemd.
De code om te werken met deze package is sterk gelijkaardig aan sklearn.
De API en een uitgebreide documentatie met voorbeeldcode kan je [hier](https://spark.apache.org/docs/latest/ml-guide.html) vinden.

Deze package bied de volgende tools aan
* ML-technieken: classificatie, regressie, clustering, ...
* Features: Extracting en transforming van features, PCA, ...
* Pipelines: Maak, train, optimaliseer en evalueer pipelines
* Persistentie: Bewaar en laden van algoritmes/modellen
* Databeheer: Algebra tools, statistieken, null-waarden, ...

Let op dat er twee API's aangeboden worden, 1 gebaseerd op RDD's en 1 op DataFrames.
De API gebaseerd op RDD's is ouder en minder flexibel dan de API gebruik makend van DataFrames.
Momenteel werken ze allebei maar in de toekomst zou de RDD gebaseerde kunnen verdwijnen.

## Utilities

### Varianten voor numpy-arrays

Voor feature sets en volledige matrices van datasets aan te maken kan je gebruik maken van de Vector en Matrix klassen.
Deze beschikken over een Dense variant waar je elk element moet ingeven of een Sparse Variant waar cellen, elementen leeg kan laten.
Dit ziet er als volgt uit:

In [6]:
from pyspark.sql import SparkSession
from pyspark.ml.linalg import Vectors
from pyspark.ml.linalg import Matrices

spark = SparkSession.builder.appName("MLLib intro").getOrCreate()

# sparse vectors
print(Vectors.sparse(4, [(3, 0.0), (1, 2.0)]))
# dit is complexer om te lezen
# sparse vector lees je als (aantal keer, het element), (aantal keer het volgende element), ...
# dit is veel efficienter als je veel keer hetzelfde element hebt

# dense vectors
print(Vectors.dense([4.0, 5.0, 6.0, 7.0]))

#matrix
print(Matrices.dense(4, 4, range(16)))
# [4,4] is de shape van de matrix -> rijen en kolommen
# range(16) is de data

(4,[1,3],[2.0,0.0])
[4.0,5.0,6.0,7.0]
DenseMatrix([[ 0.,  4.,  8., 12.],
             [ 1.,  5.,  9., 13.],
             [ 2.,  6., 10., 14.],
             [ 3.,  7., 11., 15.]])


Het is belangrijk om te weten dat dit locale datastructuren (wrapper rond numpy array) zijn en geen gedistribueerde objecten.

### Statistieken

Voor er kan gewerkt worden met statistieken moeten we (net zoals bij pandas) eerst een dataset hebben.
Hieronder maken we een random dataframe aan van 50 rijen en 4 kolommen.

In [19]:
from pyspark.mllib.random import RandomRDDs
from pyspark.ml.stat import Summarizer

sc = spark.sparkContext
                                    # 50 = rijen, 4 = kolommen
data = RandomRDDs.uniformVectorRDD(sc, 50, 4).map(lambda a: a.tolist()).toDF()
data.show()
# build in summary
data.summary().show()

#custom summary
# hierbij moet je alle features in 1 kolom gesteken, die ene kolom bevat vectoren
# dit is nodig omdat de ML-technieken ook zo werken
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=data.columns, outputCol = 'vector')
data = assembler.transform(data)
data.show(truncate=False)

summary = Summarizer.metrics('min', 'max')
data.select(summary.summary(data.vector)).show(truncate=False)
# dit geeft iets weer in de vorm van {eerste metriek: per kolom de waarde, tweede metriek: per kolom de waarde}

+--------------------+-------------------+-------------------+-------------------+
|                  _1|                 _2|                 _3|                 _4|
+--------------------+-------------------+-------------------+-------------------+
| 0.06144433047762732| 0.9834059394649531| 0.8371498653251188|   0.91506967373569|
| 0.34017123795223936| 0.3249177958463023|0.21781266422873424|0.17297541024459984|
|  0.9996874598538646| 0.1593780944677522|0.26461222718797084| 0.7960466626211362|
|  0.7051192834260629|0.28364324881697733| 0.8537541266390138|0.11161994543306109|
|  0.9531153219155979|0.12413237262448817|0.46974208760114955|  0.620303637160762|
|  0.9742607806552075| 0.9198323805537554|0.12782338610508603| 0.6101921308817179|
|  0.5490961160338745|0.06744892807445191| 0.9438273039077364| 0.5046993016195154|
| 0.21229864713840785| 0.9876989431058554| 0.5778013785548592|0.21926236026569113|
|  0.9871835219266937| 0.4333664133014189| 0.6222806229544848| 0.5871746970851577|
|   

**Correlation matrix**

Buiten de statistieken die berekend kunnen worden door de summary() functie kan ook de correlatiematrix belangrijk zijn.
Deze matrix maakt het mogelijk om het verband tussen de verscheidene features te bestuderen.
Deze matrix kan als volgt berekend worden voor een gedistribueerd dataframe.

In [27]:
from pyspark.ml.stat import Correlation
df = data.select('vector')
df.show(truncate=False)

data_corr = Correlation.corr(df, 'vector')
data_corr.collect()[0][0].values.reshape(4,4)

+---------------------------------------------------------------------------------+
|vector                                                                           |
+---------------------------------------------------------------------------------+
|[0.06144433047762732,0.9834059394649531,0.8371498653251188,0.91506967373569]     |
|[0.34017123795223936,0.3249177958463023,0.21781266422873424,0.17297541024459984] |
|[0.9996874598538646,0.1593780944677522,0.26461222718797084,0.7960466626211362]   |
|[0.7051192834260629,0.28364324881697733,0.8537541266390138,0.11161994543306109]  |
|[0.9531153219155979,0.12413237262448817,0.46974208760114955,0.620303637160762]   |
|[0.9742607806552075,0.9198323805537554,0.12782338610508603,0.6101921308817179]   |
|[0.5490961160338745,0.06744892807445191,0.9438273039077364,0.5046993016195154]   |
|[0.21229864713840785,0.9876989431058554,0.5778013785548592,0.21926236026569113]  |
|[0.9871835219266937,0.4333664133014189,0.6222806229544848,0.587174697085157

array([[1.        , 0.00422482, 0.10837841, 0.02487038],
       [0.00422482, 1.        , 0.09838635, 0.23423854],
       [0.10837841, 0.09838635, 1.        , 0.12835511],
       [0.02487038, 0.23423854, 0.12835511, 1.        ]])

**Onafhankelijksheidtest**

Naast de correlatiematrix kan het ook belangrijk zijn om de onafhankelijkheid te testen tussen elke feature en een label.
Dit kan uitgevoerd worden door een zogenaamde ChiSquareTest.
Deze krijgt als input een dataframe, de naam van de kolom met de features (als vectors) en de naam van een kolom met de labels.
We kunnen deze test uitvoeren als volgt:

In [30]:
# voeg een label-kolom toe aan df_vector
from pyspark.sql.functions import rand, when
# voor een onafhankelijkheids test hebben we een label nodig dat we proberen te voorspellen
# hier nemen we een willekeurig getal 0 of 1 (elk 50% kans)
df_label = df.withColumn('label', when(rand() > 0.5, 1).otherwise(0))
df_label.show()

from pyspark.ml.stat import ChiSquareTest
# feature kolom = vector
# label kolom = label
ind_test = ChiSquareTest.test(df_label, 'vector', 'label')
ind_test.show(truncate=False)

# pValues -> is er een significant verband tussen de feature en label, significant dat zie je dat de waarde < 0.05
# degrees of Freedom -> hoeveel van de waarden er kunnen veranderen zonder dat er een significant verband is
       # hoge waarde tov aantal rijen -> geen verband
# dit kan je gebruiken om te bepalen welke waarden/features goed gebruikt kunnen worden voor het label te voorspellen

+--------------------+-----+
|              vector|label|
+--------------------+-----+
|[0.06144433047762...|    1|
|[0.34017123795223...|    0|
|[0.99968745985386...|    0|
|[0.70511928342606...|    0|
|[0.95311532191559...|    1|
|[0.97426078065520...|    1|
|[0.54909611603387...|    1|
|[0.21229864713840...|    0|
|[0.98718352192669...|    0|
|[0.34168355767603...|    0|
|[0.98881762557951...|    1|
|[0.27595013858474...|    0|
|[0.75385193769909...|    0|
|[0.11274145386813...|    1|
|[0.88892394064739...|    0|
|[0.89571609123059...|    0|
|[1.63061743859715...|    0|
|[0.42835810872380...|    1|
|[0.70142065749526...|    0|
|[0.97987796133795...|    1|
+--------------------+-----+
only showing top 20 rows

+--------------------------------------------------------------------------------+----------------+--------------------------------------------------------------------------+
|pValues                                                                         |degreesOfFreedom|stat

**Summarizer**

Andere statistieken per kolom kunnen berekend worden door gebruik te maken van de Summarizer klasse:

In [ ]:
from pyspark.ml.stat import Summarizer
# zie hierboven

Het gebruik maken van de Summarizer maakt het dus mogelijk om rechtstreeks op de feature vectors te werken zonder ze eerst terug te moeten splitsen.

### Pipelines

Pipelines binnen Spark zijn een groep van high-level API's steunend op Dataframes om ML-pipelines aan te maken, optimaliseren en trainen.
De belangrijkste concepten binnen de Pipelines van Spark zijn:
* Dataframe: concept van de dataset
* Transformer: Zet een dataframe om in een ander dataframe
* Estimator: Zet een dataframe om in een model/transformer
* Pipeline: een ketting van transformers en estimators om een flow vast te leggen
* Parameter: API voor parameters van transformers en estimators aan te passen

Gebruik nu onderstaande mini-dataset waar we op basis van een tekstkolom met logistische regressie een bepaald label proberen te voorspellen.
Maak hiervoor een Pipeline uit die bestaat uit de volgende stappen:
* Tokenizer om de tekstkolom te splitsen in de overeenkomstige woorden
* HashingTf om de term frequency van de woorden te bepalen en het om te zetten naar een feature vector
* LogisticRegression Estimator om de voorspelling te doen.

Train daarna deze pipeline en maak de voorspellingen voor de traningsdata.
Hoe accuraat is dit model?

In [37]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer

# Prepare training documents from a list of (id, text, label) tuples.
training = spark.createDataFrame([
    (0, "a b c d e spark", 1.0),
    (1, "b d", 0.0),
    (2, "spark f g h", 1.0),
    (3, "hadoop mapreduce", 0.0)
], ["id", "text", "label"])
training.show()

# bij transformers -> ga je heel vaak wat de input kolom(s) en wat de outputkolom(s) is
# let op de outputCol nog niet bestaat
tokenizer = Tokenizer(inputCol='text', outputCol='words')
hasher = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol='features')
# hashing tf -> sparse vector met hoeveel keer elk woord voor komt
# elk woord wordt ook gehashed zodat een woord omgezet wordt naar een getal
# 262144 is het aantal woorden in de kolom / het aantal mogelijke woorden
clf = LogisticRegression(maxIter = 10, regParam=0.001)
# dit voegt 3 kolommen uit
# raw prediction -> voorspelde waarde per klasse
# probability -> kans dat het van elke klasse is
# prediction -> klasse met de hoogste kans

pipeline = Pipeline(stages=[tokenizer, hasher,clf])


# fit the pipeline
model = pipeline.fit(training)
preds = model.transform(training)
preds.show(truncate=False)

+---+----------------+-----+
| id|            text|label|
+---+----------------+-----+
|  0| a b c d e spark|  1.0|
|  1|             b d|  0.0|
|  2|     spark f g h|  1.0|
|  3|hadoop mapreduce|  0.0|
+---+----------------+-----+



+---+----------------+-----+----------------------+----------------------------------------------------------------------------+----------------------------------------+------------------------------------------+----------+
|id |text            |label|words                 |features                                                                    |rawPrediction                           |probability                               |prediction|
+---+----------------+-----+----------------------+----------------------------------------------------------------------------+----------------------------------------+------------------------------------------+----------+
|0  |a b c d e spark |1.0  |[a, b, c, d, e, spark]|(262144,[74920,89530,107107,148981,167694,173558],[1.0,1.0,1.0,1.0,1.0,1.0])|[-5.938819269022638,5.938819269022638]  |[0.0026282134969419354,0.997371786503058] |1.0       |
|1  |b d             |0.0  |[b, d]                |(262144,[89530,148981],[1.0,1.0])                    

### Evalueren van een model

In de pyspark.ml package zitten er ook functionaliteiten voor deze modellen te evalueren.
Meer informatie hierover vind je [hier](https://spark.apache.org/docs/2.2.0/mllib-evaluation-metrics.html).

In [38]:
# evalueren van het model
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator(labelCol='label', rawPredictionCol='rawPrediction')
evaluator.evaluate(preds)

1.0

### Data sources

Door gebruik te maken van de sparkContext kunnen een reeks standaard databronnen ingelezen worden om datasets uit op te bouwen (Csv, Json, ...).
Daarnaast is het ook mogelijk om een folder met een reeks beelden te gebruiken als dataset om zo een model voor image classification te trainen.
Download nu [deze](https://www.kaggle.com/returnofsputnik/chihuahua-or-muffin) dataset en upload ze naar een folder op het hadoop filesysteem.

In [39]:
import opendatasets as od

od.download("https://www.kaggle.com/returnofsputnik/chihuahua-or-muffin")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: jensbaetensodisee
Your Kaggle Key: ········


100%|████████████████████████████████████████| 183k/183k [00:00<00:00, 2.09MB/s]

In [41]:
import pydoop.hdfs as hdfs

localFS = hdfs.hdfs(host='')
client = hdfs.hdfs(host='localhost', port=9000)

if not client.exists('/user/bigdata/Spark'):
    client.create_directory('/user/bigdata/Spark')
client.set_working_directory('/user/bigdata/Spark')
print(client.working_directory())
      
# upload input.txt
localFS.copy("chihuahua-or-muffin", client, "chihuahua-or-muffin")

/user/bigdata/Spark


0

De geuploade images kunnen nu ingelezen worden als volgt:

In [46]:
df = spark.read.format('image').option('dropInvalid', True).load('Spark/chihuahua-or-muffin')
# option dropInvalid negeert bestanden die hij niet kan inlezen als figuur (anders heb je een foutmelding)
# in de load moet een directory staan bij figuren
df.select('image.origin', 'image.height', 'image.width').show(5,truncate=False)
# hierbij is er een onderliggend niveau in het dataframe
# aan de kolommen onder image kan je door gebruik te maken van . -> image.origin bvb om aan de origin kolom onder image te geraken
df.printSchema()

+----------------------------------------------------------------------------+------+------+
|origin                                                                      |height|height|
+----------------------------------------------------------------------------+------+------+
|hdfs://localhost:9000/user/bigdata/Spark/chihuahua-or-muffin/muffin-4.jpeg  |170   |170   |
|hdfs://localhost:9000/user/bigdata/Spark/chihuahua-or-muffin/muffin-7.jpeg  |172   |172   |
|hdfs://localhost:9000/user/bigdata/Spark/chihuahua-or-muffin/muffin-1.jpeg  |171   |171   |
|hdfs://localhost:9000/user/bigdata/Spark/chihuahua-or-muffin/muffin-8.jpeg  |172   |172   |
|hdfs://localhost:9000/user/bigdata/Spark/chihuahua-or-muffin/chihuahua-6.jpg|169   |169   |
+----------------------------------------------------------------------------+------+------+
only showing top 5 rows

root
 |-- image: struct (nullable = true)
 |    |-- origin: string (nullable = true)
 |    |-- height: integer (nullable = true)
 |    |--

Merk op dat het werken met images niet zo eenvoudig is.
Hiervoor wordt binnen pyspark typisch gebruik gemaakt van de [sparkdl](https://smurching.github.io/spark-deep-learning/site/api/python/sparkdl.html) package.
Hierbij staat de dl voor deep learning.
Aangezien dit ons momenteel te ver leidt ga ik dit niet verder toelichten.

Een andere aparte databron die eenvoudig ingelezen kan worden is het formaat "libsvm".
Een bestand van dit formaat wordt ingelezen als een dataframe met twee kolommen: een label en een kolom met de feature-vectors.
De code om dergelijk bestand in te laden is:

In [ ]:
df = spark.read.format("libsvm").load("{path to file here}")